In [1]:
# KNN REGRESSION
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

from math import sqrt

In [3]:
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [4]:
df = pd.read_csv("train.csv")
df.fecha = pd.to_datetime(df.fecha)

In [5]:
df.info()
#antiguedad, habitaciones, metrostotales

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

In [6]:
print("tipodepropiedad: ", df['tipodepropiedad'].isnull().sum())
print("provincia: ", df['provincia'].isnull().sum())
print("antiguedad: ", df['antiguedad'].isnull().sum())
print("habitaciones: ", df['habitaciones'].isnull().sum())
print("metrostotales: ", df['metrostotales'].isnull().sum())
print("precio: ", df['precio'].isnull().sum())

df1 = df[['tipodepropiedad','provincia','antiguedad','habitaciones','metrostotales','precio']]
df1.head()

tipodepropiedad:  46
provincia:  155
antiguedad:  43555
habitaciones:  22471
metrostotales:  51467
precio:  0


,tipodepropiedad,provincia,antiguedad,habitaciones,metrostotales,precio
0,Apartamento,Distrito Federal,nan,2.00,80.00,2273000.00
1,Casa en condominio,Distrito Federal,10.00,3.00,180.00,3600000.00
2,Casa,Jalisco,5.00,3.00,166.00,1200000.00
3,Casa,Edo. de México,1.00,2.00,67.00,650000.00
4,Apartamento,Jalisco,10.00,2.00,95.00,1150000.00


In [7]:
df2 = df[['tipodepropiedad','provincia','antiguedad','habitaciones','metrostotales','precio']]
df2


,tipodepropiedad,provincia,antiguedad,habitaciones,metrostotales,precio
0,Apartamento,Distrito Federal,nan,2.00,80.00,2273000.00
1,Casa en condominio,Distrito Federal,10.00,3.00,180.00,3600000.00
2,Casa,Jalisco,5.00,3.00,166.00,1200000.00
3,Casa,Edo. de México,1.00,2.00,67.00,650000.00
4,Apartamento,Jalisco,10.00,2.00,95.00,1150000.00
5,Apartamento,Distrito Federal,5.00,2.00,90.00,1100000.00
6,Casa,Oaxaca,nan,3.00,160.00,1150000.00
7,Casa,Quintana Roo,2.00,4.00,293.00,4200000.00
8,Apartamento,Colima,1.00,2.00,nan,310000.00
9,Terreno,Edo. de México,nan,nan,nan,6200000.00


In [8]:
# Con dummy_na=True, creamos la categoria "Es nulo" como una coordenada más de los one-hot vectors
# Comentar: ¿Hay leaks acá? ¿Sí / No? ¿Por qué?
df2 = pd.get_dummies(df2, dummy_na=True)
print(f"Cantidad de columnas después del one-hot-encoding: {len(df2.columns)}")
df2.head()

Cantidad de columnas después del one-hot-encoding: 62


,antiguedad,habitaciones,metrostotales,precio,tipodepropiedad_Apartamento,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Garage,tipodepropiedad_Hospedaje,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Villa,tipodepropiedad_nan,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_nan
0,nan,2.00,80.00,2273000.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10.00,3.00,180.00,3600000.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5.00,3.00,166.00,1200000.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.00,2.00,67.00,650000.00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10.00,2.00,95.00,1150000.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
display(df2.isnull().sum())
numeric_columns_with_nulls = list(set(df2.columns[df2.isnull().sum() > 0].tolist()) \
                                  - set(['tipodepropiedad', 'ciudad', 'provincia']))
print(numeric_columns_with_nulls)

antiguedad                                       43555
habitaciones                                     22471
metrostotales                                    51467
precio                                               0
tipodepropiedad_Apartamento                          0
tipodepropiedad_Bodega comercial                     0
tipodepropiedad_Casa                                 0
tipodepropiedad_Casa en condominio                   0
tipodepropiedad_Casa uso de suelo                    0
tipodepropiedad_Departamento Compartido              0
tipodepropiedad_Duplex                               0
tipodepropiedad_Edificio                             0
tipodepropiedad_Garage                               0
tipodepropiedad_Hospedaje                            0
tipodepropiedad_Huerta                               0
tipodepropiedad_Inmuebles productivos urbanos        0
tipodepropiedad_Local Comercial                      0
tipodepropiedad_Local en centro comercial            0
tipodeprop

['antiguedad', 'habitaciones', 'metrostotales']


In [10]:
# Para los nulls numéricos, usar un Imputer con strategy mean (reemplazamos los NaN por el promedio)
# Para no leakear, spliteamos el dataset antes
X2 = df2.drop("precio", axis=1)
y2 = df2['precio']
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.25, random_state=1)

print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)
print()
print("y_train shape:", y_train.shape)
print("y_test  shape:", y_test.shape)

X_train shape: (180000, 61)
X_test  shape: (60000, 61)

y_train shape: (180000,)
y_test  shape: (60000,)


In [11]:
X_train[['antiguedad']]

,antiguedad
193355,20.00
76494,2.00
205960,nan
65931,4.00
83352,0.00
230050,1.00
188170,0.00
38646,nan
90658,0.00
142343,10.00


In [12]:
print(X_train.isnull().sum().sum())
print(X_test.isnull().sum().sum())

print(y_train.isnull().sum().sum())
print(y_test.isnull().sum().sum())


88267
29226
0
0


In [13]:
for c in numeric_columns_with_nulls:
    imp = Imputer()
    X_train[c] = imp.fit_transform(X_train[[c]])
    X_test[c] = imp.transform(X_test[[c]])

/home/pablocruz/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/pablocruz/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/pablocruz/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [14]:
print(X_train.isnull().sum().sum())
print(X_test.isnull().sum().sum())

print(y_train.isnull().sum().sum())
print(y_test.isnull().sum().sum())


0
0
0
0


In [15]:
nX_train = (X_train - X_train.mean()) / (X_train.std())

nX_test = (X_test - X_test.mean()) / (X_test.std())

ny_train = (y_train - y_train.mean()) / (y_train.std())

ny_test = (y_test - y_test.mean()) / (y_test.std())

print(X_train.isnull().sum().sum())
print(X_test.isnull().sum().sum())

print(y_train.isnull().sum().sum())
print(y_test.isnull().sum().sum())


0
0
0
0


In [16]:
def knn_train_test(X_train, X_test, y_train, y_test, train_col):
    knn = KNeighborsRegressor(n_neighbors=7)
    
    # Fit a KNN model using default k value.
    print(X_train)
    
    knn.fit(X_train[[train_col]], y_train)
    
    # Make predictions using model.
    predicted_labels = knn.predict(X_test)
    
    # Calculate and return RMSE.
    return RMSLE(test_df[target_col], predicted_labels)

In [15]:
# Lets test a couple of predictors

# print('tipodepropiedad: ', knn_train_test(nX_train, nX_test, ny_train, ny_test, 'tipodepropiedad'))
# print('antiguedad: ', knn_train_test('antiguedad', 'precio', normalized_df))
# print('habitaciones: ', knn_train_test('habitaciones', 'precio', normalized_df))
# print('metrostotales: ', knn_train_test('metrostotales', 'precio', normalized_df))

        antiguedad  habitaciones  metrostotales  tipodepropiedad_Apartamento  \
193355        1.38         -1.06           1.65                         1.78   
76494        -0.71         -1.06          -0.00                         1.78   
205960       -0.00          1.29          -0.00                         1.78   
65931        -0.48          1.29          -0.00                        -0.56   
83352        -0.94          0.11          -0.68                        -0.56   
230050       -0.82         -1.06          -0.54                        -0.56   
188170       -0.94          0.11          -0.68                        -0.56   
38646        -0.00          1.29           1.95                        -0.56   
90658        -0.94         -0.00           2.22                        -0.56   
142343        0.22         -1.06           0.28                        -0.56   
231933       -0.00         -0.00          -0.00                        -0.56   
223481        1.38          0.11        

KeyError: "['tipodepropiedad'] not in index"

In [17]:
def knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols):

    k_values = list(range(1,15,3))
#     k_values = [1,2]
    k_rmses = {}
    
    for k in k_values:
        # Fit a KNN model using default k value.
        knn = KNeighborsRegressor(n_neighbors=k)
        knn.fit(X_train[train_cols], y_train)

        # Make predictions using model.
        predicted_labels = knn.predict(X_test[train_cols])

        # Calculate and return RMSE.
        rmse = RMSLE(y_test, predicted_labels)
        
        k_rmses[k] = rmse
    return k_rmses

In [18]:
colsTipoDePropiedad = []
for col in nX_train.columns.tolist():
    if("tipodepropiedad" in col):
        colsTipoDePropiedad.append(col)
    
colsTipoDePropiedad

['tipodepropiedad_Apartamento',
 'tipodepropiedad_Bodega comercial',
 'tipodepropiedad_Casa',
 'tipodepropiedad_Casa en condominio',
 'tipodepropiedad_Casa uso de suelo',
 'tipodepropiedad_Departamento Compartido',
 'tipodepropiedad_Duplex',
 'tipodepropiedad_Edificio',
 'tipodepropiedad_Garage',
 'tipodepropiedad_Hospedaje',
 'tipodepropiedad_Huerta',
 'tipodepropiedad_Inmuebles productivos urbanos',
 'tipodepropiedad_Local Comercial',
 'tipodepropiedad_Local en centro comercial',
 'tipodepropiedad_Lote',
 'tipodepropiedad_Nave industrial',
 'tipodepropiedad_Oficina comercial',
 'tipodepropiedad_Otros',
 'tipodepropiedad_Quinta Vacacional',
 'tipodepropiedad_Rancho',
 'tipodepropiedad_Terreno',
 'tipodepropiedad_Terreno comercial',
 'tipodepropiedad_Terreno industrial',
 'tipodepropiedad_Villa',
 'tipodepropiedad_nan']

In [20]:
print(nX_train.isnull().sum().sum())
print(nX_test.isnull().sum().sum())

print(ny_train.isnull().sum().sum())
print(ny_test.isnull().sum().sum())

nX_test.isnull().sum()

0
120000
0
0


antiguedad                                           0
habitaciones                                         0
metrostotales                                        0
tipodepropiedad_Apartamento                          0
tipodepropiedad_Bodega comercial                     0
tipodepropiedad_Casa                                 0
tipodepropiedad_Casa en condominio                   0
tipodepropiedad_Casa uso de suelo                    0
tipodepropiedad_Departamento Compartido              0
tipodepropiedad_Duplex                               0
tipodepropiedad_Edificio                             0
tipodepropiedad_Garage                           60000
tipodepropiedad_Hospedaje                        60000
tipodepropiedad_Huerta                               0
tipodepropiedad_Inmuebles productivos urbanos        0
tipodepropiedad_Local Comercial                      0
tipodepropiedad_Local en centro comercial            0
tipodepropiedad_Lote                                 0
tipodeprop

In [25]:
colsTipoDePropiedad.remove("tipodepropiedad_Garage")
colsTipoDePropiedad.remove("tipodepropiedad_Hospedaje")
colsTipoDePropiedad

['tipodepropiedad_Apartamento',
 'tipodepropiedad_Bodega comercial',
 'tipodepropiedad_Casa',
 'tipodepropiedad_Casa en condominio',
 'tipodepropiedad_Casa uso de suelo',
 'tipodepropiedad_Departamento Compartido',
 'tipodepropiedad_Duplex',
 'tipodepropiedad_Edificio',
 'tipodepropiedad_Huerta',
 'tipodepropiedad_Inmuebles productivos urbanos',
 'tipodepropiedad_Local Comercial',
 'tipodepropiedad_Local en centro comercial',
 'tipodepropiedad_Lote',
 'tipodepropiedad_Nave industrial',
 'tipodepropiedad_Oficina comercial',
 'tipodepropiedad_Otros',
 'tipodepropiedad_Quinta Vacacional',
 'tipodepropiedad_Rancho',
 'tipodepropiedad_Terreno',
 'tipodepropiedad_Terreno comercial',
 'tipodepropiedad_Terreno industrial',
 'tipodepropiedad_Villa',
 'tipodepropiedad_nan']

In [52]:
# aca filtramos 1tipo de propiedad por tener valores nulos no manejables.
print('tipodepropiedad: ', knn_train_test_mult2(X_train, X_test, y_train, y_test, colsTipoDePropiedad))


tipodepropiedad:  {1: 1.5889261693234131, 4: 1.0183374185173388, 7: 0.8604247007398801, 10: 0.8454101005085098, 13: 0.8555224099217467}


In [57]:
# aca filtramos 1tipo de propiedad por tener valores nulos no manejables.
colsTipoDePropiedad
knn_values = [50,90]
print('tipodepropiedad: ', knn_train_test_mult2(X_train, X_test, y_train, y_test, colsTipoDePropiedad, knn_values))


tipodepropiedad:  {50: 0.9299385860547783, 90: 0.8859242192473159}


In [28]:
#con knn = 10 perfomo mejor intentamos con knn= 10 para provincias.

def knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols, k_vals):

    k_values = k_vals
    k_rmses = {}
    
    for k in k_values:
        # Fit a KNN model using default k value.
        knn = KNeighborsRegressor(n_neighbors=k)
        knn.fit(X_train[train_cols], y_train)

        # Make predictions using model.
        predicted_labels = knn.predict(X_test[train_cols])

        # Calculate and return RMSE.
        rmse = RMSLE(y_test, predicted_labels)
        
        k_rmses[k] = rmse
    return k_rmses

In [29]:
colsProvincias = []
for col in nX_train.columns.tolist():
    if("provincia" in col):
        colsProvincias.append(col)
    
colsProvincias

['provincia_Aguascalientes',
 'provincia_Baja California Norte',
 'provincia_Baja California Sur',
 'provincia_Campeche',
 'provincia_Chiapas',
 'provincia_Chihuahua',
 'provincia_Coahuila',
 'provincia_Colima',
 'provincia_Distrito Federal',
 'provincia_Durango',
 'provincia_Edo. de México',
 'provincia_Guanajuato',
 'provincia_Guerrero',
 'provincia_Hidalgo',
 'provincia_Jalisco',
 'provincia_Michoacán',
 'provincia_Morelos',
 'provincia_Nayarit',
 'provincia_Nuevo León',
 'provincia_Oaxaca',
 'provincia_Puebla',
 'provincia_Querétaro',
 'provincia_Quintana Roo',
 'provincia_San luis Potosí',
 'provincia_Sinaloa',
 'provincia_Sonora',
 'provincia_Tabasco',
 'provincia_Tamaulipas',
 'provincia_Tlaxcala',
 'provincia_Veracruz',
 'provincia_Yucatán',
 'provincia_Zacatecas',
 'provincia_nan']

In [33]:
#con knn = 10 perfomo mejor intentamos con knn= 10 para provincias.
knn_values = [10]
print('provincias: ', knn_train_test_mult2(X_train, X_test, y_train, y_test, colsProvincias, knn_values))


provincias:  {10: 0.8936255230615647}


In [34]:
#pruebo con otros valores de k
knn_values = [5,15,20]
print('provincias: ', knn_train_test_mult2(X_train, X_test, y_train, y_test, colsProvincias, knn_values))


provincias:  {5: 0.9684965569473464, 15: 0.850661791970249, 20: 0.850298063676849}


In [58]:
#pruebo con otros valores de k
knn_values = [50, 90]
print('provincias: ', knn_train_test_mult2(X_train, X_test, y_train, y_test, colsProvincias, knn_values))


provincias:  {50: 0.8542124024879421, 90: 0.8507668142123376}


In [43]:
k_rmse_results = {}

knn_values = [5,15,50,90]
variables = ['antiguedad', 'habitaciones', 'metrostotales']
for var in variables:
    rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, [var], knn_values)
    k_rmse_results[var] = rmse_val

k_rmse_results

{'antiguedad': {5: 0.9539789862280427,
  15: 0.9559551407614125,
  50: 0.9017131250911609,
  90: 0.8976530956075471},
 'habitaciones': {5: 1.15147307484064,
  15: 0.9252672999781049,
  50: 0.8338103921689428,
  90: 0.8563894569824206},
 'metrostotales': {5: 0.7787011760817303,
  15: 0.7606113278957694,
  50: 0.7450767483561302,
  90: 0.7459690729247939}}

In [ ]:
#Elegimos los mejores features
#tipodepropiedad:  {1: 1.5889261693234131, 4: 1.0183374185173388, 7: 0.8604247007398801, 10: 0.8454101005085098, 13: 0.8555224099217467, {50: 0.9299385860547783, 90: 0.8859242192473159}
#provincias:  {5: 0.9684965569473464, 10: 0.8936255230615647, 15: 0.850661791970249, 20: 0.850298063676849, 50: 0.8542124024879421, 90: 0.8507668142123376}
#'antiguedad': {5: 0.9539789862280427,15: 0.9559551407614125, 50: 0.9017131250911609, 90: 0.8976530956075471}
#'habitaciones': {5: 1.15147307484064, 15: 0.9252672999781049, 50: 0.8338103921689428, 90: 0.8563894569824206},
#'metrostotales': {5: 0.7787011760817303, 15: 0.7606113278957694, 50: 0.7450767483561302, 90: 0.7459690729247939}}

In [66]:
#elegimos el mejor knn = 90
#vamos sumando features de a uno acorde al mejor resultado.
k_rmse_results = {}

In [67]:
train_cols_2 = ['metrostotales'] + colsProvincias
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_2, [90])
k_rmse_results["two best features"] = rmse_val

In [68]:

train_cols_3 = ['metrostotales'] + colsProvincias + ['habitaciones']
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_3, [90])
k_rmse_results["three best features"] = rmse_val

In [69]:
train_cols_4 = ['metrostotales'] + colsProvincias + ['habitaciones'] + colsTipoDePropiedad
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_4, [90])
k_rmse_results["four best features"] = rmse_val

In [70]:
train_cols_5 = ['metrostotales'] + colsProvincias + ['habitaciones'] + colsTipoDePropiedad + ['antiguedad']
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_5, [90])
k_rmse_results["five best features"] = rmse_val

In [71]:
k_rmse_results

{'two best features': {90: 0.6735469045143808},
 'three best features': {90: 0.6462914618452639},
 'four best features': {90: 0.6073117155279684},
 'five best features': {90: 0.6324624980516879}}

In [74]:
#elegimos el 2do mejor knn = 50
#vamos sumando features de a uno acorde al mejor resultado.
k_rmse_results2 = {}
knn_vals = [5, 10, 15, 30, 50,70]

In [75]:
train_cols_2 = ['metrostotales'] + colsProvincias
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_2, knn_vals)
k_rmse_results2["two best features"] = rmse_val

In [77]:
train_cols_3 = ['metrostotales'] + colsProvincias + ['habitaciones']
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_3, knn_vals)
k_rmse_results2["three best features"] = rmse_val

In [78]:
train_cols_4 = ['metrostotales'] + colsProvincias + ['habitaciones'] + colsTipoDePropiedad
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_4, knn_vals)
k_rmse_results2["four best features"] = rmse_val

In [79]:
train_cols_5 = ['metrostotales'] + colsProvincias + ['habitaciones'] + colsTipoDePropiedad + ['antiguedad']
rmse_val = knn_train_test_mult2(X_train, X_test, y_train, y_test, train_cols_5, knn_vals)
k_rmse_results2["five best features"] = rmse_val